<a href="https://colab.research.google.com/github/Zamoca42/TIL/blob/main/Python/Dynamic_Crawling_Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 동적 크롤링 - 다나와 PC견적

## 소개

- 다나와 PC 견적페이지에서 부품을 선택해서 견적을 맞춰보는 프로젝트
- 2023.1.21 기준 강의내용과 다름 
  - CPU 종류 선택부분부터 셀렉박스 -> 체크박스로 변경된 부분
  - html 소스에서 iframe 삭제
- 변경된 부분에 맞춰서 코드작성
- vscode로 작성,실행하고 colab으로 기록함

### 선행코드

In [ ]:
# 필요한 라이브러리

import time
import pyperclip
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 드라이버 자동 업데이트
# pip install webdriver_manager
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

- 앞선 프로젝트와 같은 크롬 드라이버와 wait코드 미리 작성

In [ ]:
chrome = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

wait = WebDriverWait(chrome, 10)
short_wait = WebDriverWait(chrome, 3)

- 자주사용할 함수를 미리 작성해준다
  - CSS선택자를 이용한 Explicit wait
  - find_elements를 이용한 요소찾기

In [ ]:
def find_present(css):
    return wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))

def finds_present(css):
    find_present(css)
    return chrome.find_elements(By.CSS_SELECTOR, css)

def find_visible(css):
    return wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, css)))

def finds_visible(css):
    find_visible(css)
    return chrome.find_elements(By.CSS_SELECTOR, css)

In [ ]:
# 다나와 PC 견적페이지
chrome.get('https://shop.danawa.com/virtualestimate/?controller=estimateMain&methods=index&marketPlaceSeq=16')

## 카테고리 선택

### CPU 카테고리 클릭

- 먼저 iframe을 이동해주어야하지만 다나와 업데이트로 iframe이 삭제


In [ ]:
# frame 이동
# find_visible("iframe#ifrmWish")
# chrome.switch_to.frame("ifrmWish")

![스크린샷 2023-01-21 오전 12 17 37](https://user-images.githubusercontent.com/96982072/213733834-8d77841a-bf49-403c-a360-db5b76f6d371.png)

- 다나와 PC주요부품에서 차례로 부품선택
  - CPU
  - 메인보드
  - 메모리
  - 그래픽카드
  - ssd
  - 케이스
  - 파워



- CPU의 element는 F12를 눌러서 확인해보면 
`<dd class="category_873 pd_item select">`
로 category_873
- 메인보드의 element는 `<dd class="category_875 pd_item">`로 category_875
- 하나하나씩 입력해줄 수 있지만 반복하는 것은 좋지 않음
  - 카테고리를 맵핑

In [ ]:
# 카테고리 맵핑 테이블
category = {
    "cpu" : "873",
    "메인보드" : "875",
    "메모리" : "874",
    "그래픽카드" : "876",
    "ssd" : "32617",
    "케이스" : "879",
    "파워" : "880",
}

# 카테고리 클릭
mainboard = find_visible("dd.category_" + category["메인보드"] + " a")
mainboard.click()

- 이렇게 카테고리별로 작성할 수 있으나 dict comprehension을 이용
  - `{ key:value for 원소 in 반복가능한 객체 }`
  - `{ key:value for 원소 in 반복가능한 객체 if문 }`
  - 한줄로 작성해 혹시 모를 오타 방지

In [ ]:
# 카테고리 맵핑 테이블
category = {
    "cpu" : "873",
    "메인보드" : "875",
    "메모리" : "874",
    "그래픽카드" : "876",
    "ssd" : "32617",
    "케이스" : "879",
    "파워" : "880",
}

# dict comprehension
category_css = {
    c: "dd.category_" + category[c] + " a" for c in category
}

find_visible(category_css["cpu"]).click()
time.sleep(1)

## CPU 제조사 선택

### 필터링

![스크린샷 2023-01-21 오전 12 28 49](https://user-images.githubusercontent.com/96982072/213736170-dbdd311d-2464-402d-b6a0-990d8ccddd0c.png)

- 강의내용에서는 셀렉박스였으나 2023.1.21 기준 체크박스로 변경
- 제조사에서 인텔 or AMD를 선택하고 제조사에 맞는 CPU 종류를 고르는 순서

### 체크박스 선택

<img width="859" alt="스크린샷 2023-01-21 오전 12 32 31" src="https://user-images.githubusercontent.com/96982072/213737990-7fbd1be8-5f06-4554-84e6-ab9603bf3376.png">

- 체크박스에서 인텔을 클릭하려면 span을 선택해 클릭해야함
  - 특이한 패턴으로 input의 `name=makerCode`가 있음
  - `+`선택자로 형제노드인 span으로 이동가능


In [ ]:
# cpu 제조사 불러오기
options = finds_visible("input[name=makerCode]+span")

for o in options:
     print(o.text)

- 출력
  
  ![스크린샷 2023-01-21 오전 12 35 54](https://user-images.githubusercontent.com/96982072/213740389-40bbf81a-a6d0-4723-aa19-9b632b321a55.png)


In [ ]:
finds_visible("input[name=makerCode]+span")[0].click()

- 리스트로 받기 때문에 0번 인덱스에 `.click()`하면 인텔에 체크

### 제조사 입력 후 선택

- 동적인 동작을 하기 위해 인텔 혹은 AMD로 입력을 받음

In [ ]:
def choose_one(text, options):
    print("-"*10)
    print(text)
    print("-"*10)
    for i in range(len(options)):
        print(f"{i+1}. {options[i]}")
    choose = input("-> ")
    return int(choose) - 1

# cpu 제조사 불러오기
options = finds_visible("input[name=makerCode]+span")
i = choose_one("cpu 제조사를 골라주세요",[x.text for x in options])
print(options[i].text)
options[i].click()

- `choose_one` 함수를 작성해 1.에는 인텔 2.에는 AMD로 입력받음
  - 1인 인텔로 입력받았지만 인텔은 0번 인덱스에 들어가있으므로
  반환값에 -1을 한다


- 결과  

  ![스크린샷 2023-01-21 오전 12 41 05](https://user-images.githubusercontent.com/96982072/213742399-d1b48621-62c2-4f4a-b309-c5c87397311c.png)


## CPU 종류 선택

### 문제점1

<img width="900" alt="스크린샷 2023-01-21 오전 12 48 34" src="https://user-images.githubusercontent.com/96982072/213743032-94f25861-762d-44c6-ad7e-a514af34b528.png">

- 제조사 이하 모든 체크박스가 `name="attribute"`을 가짐
- key로 제조사를 나누기 힘듬
- 해결방법
  - 특정 패턴을 찾기 어려웠기 때문에 제일 많은 단어를 포함한 것을 추출

In [ ]:
# cpu 종류 불러오기
title = ""
if i == 0:
    title = "코어"
elif i == 1:
    title = "라이젠"

options = finds_visible(f"input[name='attribute'][data^='{title}']+span")
i = choose_one("CPU 종류를 선택해 주세요", [x.text for x in options])
options[i].click()

- 결과  

  ![스크린샷 2023-01-21 오전 12 52 28](https://user-images.githubusercontent.com/96982072/213743919-990bca1b-9861-4b9d-a2f8-3b2d17f44706.png)

- 문제점
  - 모든 체크박스가 view되지 않았기 때문에 나머지 옵션은 빈칸

- 해결방법
  - 해당제조사의 옵션더보기 또한 인덱스와 같으므로 옵션더보기도 클릭하는 명령어를 추가

In [ ]:
# cpu 종류 불러오기
title = ""
if i == 0:
    title = "코어"
elif i == 1:
    title = "라이젠"

# 옵션 더보기
finds_visible("div.search_option_list button")[i].click()
# 제조사 선택
options = finds_visible(f"input[name='attribute'][data^='{title}']+span")
i = choose_one("CPU 종류를 선택해 주세요", [x.text for x in options])
options[i].click()

- 결과  

  ![스크린샷 2023-01-21 오전 12 55 09](https://user-images.githubusercontent.com/96982072/213744603-65b7975c-d306-4bbd-89bb-718b2f64f402.png)

- 많은 목록을 불러올 수 있게됨

- 작동 화면

  ![스크린샷 2023-01-21 오전 12 59 13](https://user-images.githubusercontent.com/96982072/213745524-c849e25f-0349-458d-9149-522b9ea1e0a5.png)

- 문제점
  - 포함되지 않은 체크박스가 있다면 그 숫자만큼 인덱스가 밀린다
    - 펜티엄 골드를 체크할 수 없음

- 해결방법  
  
  <img width="884" alt="스크린샷 2023-01-21 오전 1 58 08" src="https://user-images.githubusercontent.com/96982072/213757915-13b330b8-d5ad-401e-8251-c36ea62f0316.png">
  
  - value값의 숫자가 카테고리별로 공통된 것을 발견
    - 인텔 CPU종류의 attribute 중 value값을 `873|40|`포함하는 값으로 변경  
  




  

In [ ]:
# cpu 종류 불러오기
value = ""
if i == 0:
    # 인텔
    value = "873|40|"
elif i == 1:
    # 라이젠
    value = "873|312287|"
finds_visible("div.search_option_list button")[i].click()
options = finds_visible(f"input[name='attribute'][value^='{ value }']+span")
#options = finds_visible(f"input[name='attribute'][data^='{ title }']+span")
i = choose_one("CPU 종류를 선택해 주세요", [x.text for x in options])
options[i].click()

time.sleep(10)

chrome.quit()

- 결과  

  ![스크린샷 2023-01-21 오전 1 59 45](https://user-images.githubusercontent.com/96982072/213759818-c5fa10b0-4237-458f-b474-79ad2424ec40.png)

## 최종코드

1. 반복되는 코드는 함수로 바꿈
  - 제품 선택에서 `value='876|655|'`과 같이 `value='category|menu|'`로 반복되는 패턴 발견
    - 추가적인 함수화 가능
2. Message: stale element reference: element is not attached to the page document 오류 해결
  - 목록을 받아오는 로딩시간이 길어질 수 있으므로 `time.sleep`으로 시간을 벌어줌
  - https://moondol-ai.tistory.com/239
3. 메인보드 이후로 반복되는 코드이기 때문에 그래픽카드까지만 작성


In [ ]:
# 그래픽카드 제품시리즈
finds_visible("div.search_option_list button")[1].click()
options = finds_visible("input[value^='876|655|']+span")
i = choose_one("제품시리즈를 선택해주세요", [x.text for x in options])
options[i].click()

In [ ]:
import time
import pyperclip
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 드라이버 자동 업데이트
# pip install webdriver_manager
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 카테고리 맵핑 테이블
category = {
    "cpu" : "873",
    "메인보드" : "875",
    "메모리" : "874",
    "그래픽카드" : "876",
    "ssd" : "32617",
    "케이스" : "879",
    "파워" : "880",
}

# dict comprehension
category_css = {
    c: "dd.category_" + category[c] + " a" for c in category
}

chrome = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

wait = WebDriverWait(chrome, 10)
short_wait = WebDriverWait(chrome, 3)

def find_present(css):
    return wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))

def finds_present(css):
    find_present(css)
    return chrome.find_elements(By.CSS_SELECTOR, css)

def find_visible(css):
    return wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, css)))

def finds_visible(css):
    find_visible(css)
    return chrome.find_elements(By.CSS_SELECTOR, css)

def choose_one(text, options):
    print("-"*10)
    print(text)
    print("-"*10)
    for i in range(len(options)):
        print(f"{i+1}. {options[i]}")
    choose = input("-> ")
    return int(choose) - 1

def parse_products():
    time.sleep(2)
    products = []
    for p in finds_visible("div.scroll_box tr[class^=productList_]"):
        name = p.find_element(By.CSS_SELECTOR, "p.subject a").text
        try:
            price = p.find_element(By.CSS_SELECTOR, "span.prod_price").text
        except:
            continue
        products.append((name, price))
    return products

def go_to_category(category_name):
    find_visible(category_css[category_name]).click()
    time.sleep(1)
# print(category_css)

def choose_maker(text):
    options = finds_visible("input[name=makerCode]+span")
    try: 
        finds_visible("div.search_option_list button")[0].click()
    except:
        pass
    
    i = choose_one(text,[x.text for x in options])
    print(options[i].text)
    # finds_visible("input[name=makerCode]+span")[0].click()
    options[i].click()
    return i

# 다나와 PC 견적
chrome.get('https://shop.danawa.com/virtualestimate/?controller=estimateMain&methods=index&marketPlaceSeq=16')

###### cpu #######

# 카테고리 클릭
go_to_category("cpu")

# 제조사 불러오기
options = finds_visible("input[name=makerCode]+span")
maker_idx = choose_one("cpu 제조사를 골라주세요",[x.text for x in options])
print(options[maker_idx].text)
options[maker_idx].click()

# 종류 불러오기
is_intel = False
is_amd = False
value = ""
if maker_idx == 0:
    # 인텔
    is_intel = True
    value = "873|40|"
elif maker_idx == 1:
    # 라이젠
    is_amd = True
    value = "873|312287|"

finds_visible("div.search_option_list button")[maker_idx].click()
options = finds_visible(f"input[name='attribute'][value^='{ value }']+span")
#options = finds_visible(f"input[name='attribute'][data^='{ title }']+span")
i = choose_one("CPU 종류를 선택해 주세요", [x.text for x in options])
options[i].click()

# 목록 선택하기

cpus = parse_products()
# for cpu in cpus:
#     print(cpu)

###### 메인보드 #######

# 카테고리
go_to_category("메인보드")

# 제조사
choose_maker("메인보드 제조사를 골라주세요")

options = finds_visible("input[value^='875|499|']+span")
# 종류
if is_intel:
    # 인텔
    options[0].click()
elif is_amd:
    # 라이젠
    options[1].click()


# 목록 선택
mainboards = parse_products()

###### 메모리 #######

# 카테고리
go_to_category("메모리")

# 제조사
choose_maker("메모리 제조사를 골라주세요")

# 데스크탑
finds_visible("input[value^='874|278|']+span")[0].click()

# DDR4
finds_visible("input[value^='874|277|']+span")[1].click()

# 메모리 용량
finds_visible("div.search_option_list button")[2].click()
options = finds_visible("input[value^='874|282|']+span")
i = choose_one("메모리 용량을 선택해주세요", [x.text for x in options])
options[i].click()

# 목록 선택
memories = parse_products()

###### 그래픽카드 #######

# 카테고리
go_to_category("그래픽카드")

# 제조사
choose_maker("그래픽카드 제조사를 골라주세요")

# 칩셋
options = finds_visible("input[value^='876|654|']+span")
i = choose_one("칩셋 제조사를 선택해주세요", [x.text for x in options])
options[i].click()

# 제품시리즈
finds_visible("div.search_option_list button")[1].click()
options = finds_visible("input[value^='876|655|']+span")
i = choose_one("제품시리즈를 선택해주세요", [x.text for x in options])
options[i].click()

# 목록 선택
graphics = parse_products()

# cpus, mainboards, memories, graphics

popular = {
    "cpu": cpus[0],
    "mainboard": mainboards[0],
    "memory" : memories[0], 
    "graphic" : graphics[0]
}
print("-"*50)
print("인기 1위 조합 입니다")
print("cpu : ", popular["cpu"])
print("mainboard : ", popular["mainboard"])
print("memory : ", popular["memory"])
print("graphic : ", popular["graphic"])


def find_cheap(arr):
    cheap_idx = 0
    for i in range(len(arr)):
        cheap = arr[cheap_idx]
        a = arr[i]
        if int(a[1].replace(',','')) < int(cheap[1].replace(',','')):
            cheap_idx = i
    return arr[cheap_idx]

# 가장 가성비
recommend = {
    "cpu": find_cheap(cpus),
    "mainboard": find_cheap(mainboards),
    "memory" : find_cheap(memories), 
    "graphic" : find_cheap(graphics)
}
print("-"*50)
print("가성비 1위 조합 입니다")
print("cpu :", recommend["cpu"])
print("mainboard :", recommend["mainboard"])
print("memory :", recommend["memory"])
print("graphic :", recommend["graphic"])

time.sleep(10)

chrome.quit()

- 결과  

  ![스크린샷 2023-01-21 오후 9 43 37](https://user-images.githubusercontent.com/96982072/213867511-9f14b6e3-ce5c-4806-a647-792d318b8287.png)
